# Lab 3 - Test the Event-Driven Agentic Workflow

In this notebook, we will develop an event-driven workflow so that whenever a new ticket is created in the `TicketTable`, a ticket agent is automatically invoked. When the agent resolves the ticket or assigns it to someone else, an email confirmation will be sent.

1. Notebook Setup 
2. Create Amazon SNS Topic and subscrive to it via email
3. Modify ProcessTicketDynamoDBStreamFunction AWS Lambda Function to handle events
4. Test Events

![event-driven-flow](images/architecture-event-driven.png)

We will start by creating a SNS topic to receive email when ticket is resolved or assigned to someone. Then we will modify the code for `ProcessTicketDynamoDBStreamFunction` AWS Lambda function to hanlde INSERT and MODIFY events. 

## 1. Notebook Setup 

In [ ]:
%store -r

In [ ]:
import boto3
from time import sleep
import zipfile
from io import BytesIO
from utility.knowledgebase import interactive_sleep

# Create an SNS client
sns = boto3.client('sns')
lambda_client = boto3.client('lambda')
sts_client = boto3.client('sts')

region_name = boto3.Session().region_name
account_id = sts_client.get_caller_identity()["Account"]
region_name, account_id

In [ ]:
sns_topic_name = 'NotifyTicketCreation'
participant_email_address = '' # OPTIONAL TODO: provide participant email

<div class="alert alert-block alert-warning">
<b>Email:</b> Providing your email for SNS notifications is optional. If you choose not to provide an email address, you will not receive email notifications when your ticket is processed by an agent. However, this is not a crucial requirement for this workshop.
</div>

In [ ]:
email_functionality = False
try:
    assert participant_email_address != '' # This is OPTIONAL
    
    print("Email functionality will be created")
    email_functionality = True
except Exception as ex:
    print("Email functionality will not be created")
    email_functionality = False

## OPTIONAL: 2. Create Amazon SNS Topic and subscrive to it via email

In [ ]:
topic_arn = None
if email_functionality:
    topic_response = sns.create_topic(Name=sns_topic_name)
    topic_arn = topic_response['TopicArn']


    # Subscribe an email address to the topic
    interactive_sleep(30)
    subscription_response = sns.subscribe(
        TopicArn=topic_arn,
        Protocol='email',
        Endpoint= participant_email_address,

        ReturnSubscriptionArn=True
    )

    subscription_arn = subscription_response['SubscriptionArn']
    print(f'Subscription ARN: {subscription_arn}')

<div class="alert alert-block alert-warning">
<b>Confirm SNS subscription:</b> You will receive an email in your inbox, make sure to confirm SNS subscription.
</div>

![sns](images/sns-confirm.png)

<div class="alert alert-block alert-warning">
<b>Confirm SNS subscription:</b> You will receive an email in your inbox, make sure to confirm SNS subscription.
</div>

In [ ]:
%store topic_arn sns_topic_name

## 3. Modify ProcessTicketDynamoDBStreamFunction AWS Lambda Function to handle events

<div class="alert alert-block alert-warning">
<b> SNS Email functionality:</b> Uncomment the logic in the following cell to implement email functionality. Uncomment only if you have successfully completed the optional section above to create an SNS subscription.
</div>

In [ ]:
%%writefile index.py
import json
import boto3
import uuid
import os

bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
sns_client = boto3.client('sns')

region_name = os.environ['region_name']
account_id = os.environ['account_id']
agentId = os.environ['agent_id']
knowledgeBaseId = os.environ['kb_id']

def lambda_handler(event, context):
    for record in event['Records']:
        if record['eventName'] == 'INSERT':
          
            employeeId = record["dynamodb"]["NewImage"]["employeeId"]["S"]
            ticketId = record["dynamodb"]["NewImage"]["ticketId"]["S"]
            ticket = record["dynamodb"]["NewImage"]["ticket_content"]["S"]

            inputText = f"""
            Please either auto-resolve the ticket or assign it to environment owner:

            <ticket>
            {ticket}
            </ticket>
            
            Ensure to resolve the ticket by calling TicketAPI. 
            """
            print(inputText)
            sessionId = str(uuid.uuid1())

            response = bedrock_agent_runtime_client.invoke_agent(
                              inputText=inputText,
                              agentId=agentId, # Agent ID
                              agentAliasId="TSTALIASID", 
                              sessionId=sessionId,
                              enableTrace=True, 
                              endSession=False,
                              sessionState={
                                  "sessionAttributes": {
                                      "employeeId": employeeId,
                                      "ticketId": ticketId
                                  },
                                  "knowledgeBaseConfigurations": [
                                    {
                                        'knowledgeBaseId': knowledgeBaseId,
                                        'retrievalConfiguration': {
                                            'vectorSearchConfiguration': {
                                                'numberOfResults': 3,
                                                'overrideSearchType': 'HYBRID'
                                            }
                                        }
                                    },
                                ]
                              }
                          )
            print(response)

        elif record['eventName'] == 'MODIFY':
            
            ###########################################################
            ## Uncomment this logic if you want email functionality ##
            #########################################################
            
            # ticketId = record["dynamodb"]["NewImage"]["ticketId"]["S"]
            # assignStatus = record["dynamodb"]["NewImage"]["assignStatus"]["S"]
            # communication = record["dynamodb"]["NewImage"]["communication"]["S"]
            # ticket_content = record["dynamodb"]["NewImage"]["ticket_content"]["S"]
            # message = f"""\n Update to Ticket {ticketId}\n\n\nassignStatus: {assignStatus}\n\n\ncommunication: {communication}\n\n\nticket_content: {ticket_content}"""
            # print(message)
            # response = sns_client.publish(
            #     TopicArn=f'arn:aws:sns:{region_name}:{account_id}:NotifyTicketCreation', # SNS Topic,
            #     Message=message,
            # )
            
            ###########################################################
            ## Uncomment this logic if you want email functionality ##
            #########################################################
            pass

    return {
        'statusCode': 200,
        'body': json.dumps('Data processing completed successfully')
    }


In [ ]:
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("index.py")
z.close()
zip_content = s.getvalue()

response = lambda_client.update_function_code(
    FunctionName='ProcessTicketDynamoDBStreamFunction',
    ZipFile= zip_content,
    Publish=True  # Set to True to publish a new version of the function
)

interactive_sleep(30)

In [ ]:
# Add more environment variables as needed
environment_variables = {
    'region_name': region_name,
    'account_id': account_id,
    'agent_id': agent_id,
    'kb_id': kb_id,
}

In [ ]:
lambda_client.update_function_configuration(
    FunctionName='ProcessTicketDynamoDBStreamFunction',
    Environment={
        'Variables': environment_variables
    }
)

## 4. Test Event

In [ ]:
from utility.ticket import create_ticket, display_table

### 4.1 Test case 1

In [ ]:
display_table('UserAccessTable')

In [ ]:
display_table('TicketTable')

You can `employee 111` already has access to `environment 1`

In [ ]:
ticket = """
Title: Request Environment Access
EnvironmentId: 1
Buisness Justification: Need to test new features on Quicksight Dashboard
Access duration: 8 days
Access Type: Read
"""
employeeId = '111'

In [ ]:
ticketId = create_ticket(ticket, employeeId)

<div class="alert alert-block alert-info">
<b>Information:</b> Access to the employee can be auto-assigned if all the given scenarios are true:

1. The environment is owned by the manager of the employee.
2. The employee has asked for less than 30 days of access.
3. The access type is not Admin.
</div>

<div class="alert alert-block alert-warning">
<b>Warning:</b> Wait to receive an email before checking the output of `display_table('TicketTable')`
</div>

This ticket should be auto-resolved as `employee 111` already has access to `environment 1`

In [ ]:
display_table('TicketTable')

![output-test-case-1](images/output-test-case-1.png)

<div class="alert alert-block alert-success">
<b>Disclaimer:</b> Please note that the output of Large Language Models (LLMs) is non-deterministic, meaning that the responses generated may vary each time the model is run. Due to this inherent nature of LLMs, there is a possibility of observing undesirable or unexpected behavior. If such a situation arises, we recommend running the `create_ticket` function again to obtain a new response.

To enhance the resilience and robustness of this application, we strongly recommend implementing the best practices outlined in the following resources:

1. **[Best Practices for Building Robust Generative AI Applications with Amazon Bedrock Agents - Part 1](https://aws.amazon.com/blogs/machine-learning/best-practices-for-building-robust-generative-ai-applications-with-amazon-bedrock-agents-part-1/)**
2. **[Best Practices for Building Robust Generative AI Applications with Amazon Bedrock Agents - Part 2](https://aws.amazon.com/blogs/machine-learning/best-practices-for-building-robust-generative-ai-applications-with-amazon-bedrock-agents-part-2/)**
</div>


### 4.2 Test case 2

In [ ]:
ticket = """
Title: Request Environment Access
EnvironmentId: 4
Buisness Justification: Need to view model results for sales forcast Q4
Access duration: 24 days
Access Type: Read
"""
employeeId = '121'

In [ ]:
ticketId = create_ticket(ticket, employeeId)

<div class="alert alert-block alert-info">
<b>Information:</b> Access to the employee can be auto-assigned if all the given scenarios are true:

1. The environment is owned by the manager of the employee.
2. The employee has asked for less than 30 days of access.
3. The access type is not Admin.
</div>

<div class="alert alert-block alert-warning">
<b>Warning:</b> Wait to receive an email before checking the output of `display_table('TicketTable')`
</div>

Lets evaluate the above scenarios for this test case:

1. `@susi` is owner of `environment 4` and `employee 121`'s manager is `@sam`.

This ticket should be assigned to `@susi`. 

In [ ]:
display_table('TicketTable')

![output-test-case-2](images/output-test-case-2.png)

<div class="alert alert-block alert-success">
<b>Disclaimer:</b> Please note that the output of Large Language Models (LLMs) is non-deterministic, meaning that the responses generated may vary each time the model is run. Due to this inherent nature of LLMs, there is a possibility of observing undesirable or unexpected behavior. If such a situation arises, we recommend running the `create_ticket` function again to obtain a new response.

To enhance the resilience and robustness of this application, we strongly recommend implementing the best practices outlined in the following resources:

1. **[Best Practices for Building Robust Generative AI Applications with Amazon Bedrock Agents - Part 1](https://aws.amazon.com/blogs/machine-learning/best-practices-for-building-robust-generative-ai-applications-with-amazon-bedrock-agents-part-1/)**
2. **[Best Practices for Building Robust Generative AI Applications with Amazon Bedrock Agents - Part 2](https://aws.amazon.com/blogs/machine-learning/best-practices-for-building-robust-generative-ai-applications-with-amazon-bedrock-agents-part-2/)**
</div>


### 4.3 Test case 3

In [ ]:
ticket = """
Title: Request Environment Access
EnvironmentId: 1
Buisness Justification: Need to test new features on Quicksight Dashboard
Access duration: 24 days
Access Type: Read
"""

employeeId = '121'

In [ ]:
ticketId = create_ticket(ticket, employeeId)

<div class="alert alert-block alert-info">
<b>Information:</b> Access to the employee can be auto-assigned if all the given scenarios are true:

1. The environment is owned by the manager of the employee.
2. The employee has asked for less than 30 days of access.
3. The access type is not Admin.
</div>

Lets evaluate the above scenarios for this test case:

1. `@sam` is owner of `environment 1` and `employee 121`'s manager is also `@sam`.
2. The employee has asked for less than 30 days of access.
3. The access type is not Admin.

This ticket should be auto-resolved.

<div class="alert alert-block alert-warning">
<b>Warning:</b> Wait to receive an email before checking the output of `display_table('TicketTable')`
</div>

In [ ]:
display_table('TicketTable')

![output-test-case-3](images/output-test-case-3.png)

<div class="alert alert-block alert-success">
<b>Disclaimer:</b> Please note that the output of Large Language Models (LLMs) is non-deterministic, meaning that the responses generated may vary each time the model is run. Due to this inherent nature of LLMs, there is a possibility of observing undesirable or unexpected behavior. If such a situation arises, we recommend running the `create_ticket` function again to obtain a new response.

To enhance the resilience and robustness of this application, we strongly recommend implementing the best practices outlined in the following resources:

1. **[Best Practices for Building Robust Generative AI Applications with Amazon Bedrock Agents - Part 1](https://aws.amazon.com/blogs/machine-learning/best-practices-for-building-robust-generative-ai-applications-with-amazon-bedrock-agents-part-1/)**
2. **[Best Practices for Building Robust Generative AI Applications with Amazon Bedrock Agents - Part 2](https://aws.amazon.com/blogs/machine-learning/best-practices-for-building-robust-generative-ai-applications-with-amazon-bedrock-agents-part-2/)**
</div>


<div class="alert alert-block alert-warning">
<b>Next steps:</b> Remember to run the CLEANUP notebook at the end of your session.
</div>